<a href="https://colab.research.google.com/github/yunseo-qleap/aeqis/blob/inhyuk/noise_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

9 qubit error model 


In [34]:
import pennylane as qml
from pennylane import numpy as np

In [40]:
dev = qml.device('default.mixed', wires=9)


In [41]:
ev = np.tensor(0.7781, requires_grad=False)  # observed expectation value

def sigmoid(x):
    return 1/(1+np.exp(-x))

@qml.qnode(dev)
def damping_circuit(x):
    qml.Hadamard(wires=0)
    for i in range(8):
       qml.CNOT(wires=[i, i+1])
       qml.AmplitudeDamping(sigmoid(x), wires=i)  # p = sigmoid(x) 
    return qml.expval(qml.PauliZ(0) @ qml.PauliZ(1))

In [42]:
def cost(x, target):
    return (damping_circuit(x) - target)**2

In [43]:
opt = qml.GradientDescentOptimizer(stepsize=10)
steps = 35
x = np.tensor(0.0, requires_grad=True)

for i in range(steps):
    (x, ev), cost_val = opt.step_and_cost(cost, x, ev)
    if i % 5 == 0 or i == steps - 1:
        print(f"Step: {i}    Cost: {cost_val}")

print(f"QNode output after optimization = {damping_circuit(x):.4f}")
print(f"Experimental expectation value = {ev}")
print(f"Optimized noise parameter p = {sigmoid(x.take(0)):.4f}")

/usr/local/lib/python3.7/dist-packages/autograd/numpy/numpy_wrapper.py:156: ComplexWarning: Casting complex values to real discards the imaginary part
  return A.astype(dtype, order, casting, subok, copy)


Step: 0    Cost: 0.07733961000000007
Step: 5    Cost: 0.07733961000000007
Step: 10    Cost: 0.07733961000000007
Step: 15    Cost: 0.07733961000000007
Step: 20    Cost: 0.07733961000000007
Step: 25    Cost: 0.07733961000000007
Step: 30    Cost: 0.07733961000000007
Step: 34    Cost: 0.07733961000000007
QNode output after optimization = 0.5000
Experimental expectation value = 0.7781
Optimized noise parameter p = 0.5000


In [44]:
print(f"Output state is = \n{np.real(dev.state)}")

Output state is = 
[[0.5        0.         0.         ... 0.         0.         0.03125   ]
 [0.         0.00195313 0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.00195313 0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.03125    0.         0.         ... 0.         0.         0.00195313]]


9큐빗 비트플립 모델 

In [45]:
import matplotlib.pyplot as plt
fig = qml.draw_mpl(dev)
plt.show()

In [46]:
@qml.qnode(dev)
def bitflip_circuit(p):
    qml.Hadamard(wires=0)
    for i in range(8):
      qml.CNOT(wires=[i, i+1])
      qml.BitFlip(p, wires=i)

    return qml.expval(qml.PauliZ(0) @ qml.PauliZ(1))


ps = [0.001, 0.01, 0.1, 0.2]
for p in ps:
    print(f"QNode output for bit flip probability {p} is {bitflip_circuit(p):.4f}")

QNode output for bit flip probability 0.001 is 0.9960
QNode output for bit flip probability 0.01 is 0.9604
QNode output for bit flip probability 0.1 is 0.6400
QNode output for bit flip probability 0.2 is 0.3600


In [47]:
print(f"Output state for bit flip probability {p} is \n{np.real(dev.state)}")

Output state for bit flip probability 0.2 is 
[[8.388608e-02 0.000000e+00 0.000000e+00 ... 0.000000e+00 0.000000e+00
  8.388608e-02]
 [0.000000e+00 1.280000e-06 0.000000e+00 ... 0.000000e+00 1.280000e-06
  0.000000e+00]
 [0.000000e+00 0.000000e+00 2.097152e-02 ... 2.097152e-02 0.000000e+00
  0.000000e+00]
 ...
 [0.000000e+00 0.000000e+00 2.097152e-02 ... 2.097152e-02 0.000000e+00
  0.000000e+00]
 [0.000000e+00 1.280000e-06 0.000000e+00 ... 0.000000e+00 1.280000e-06
  0.000000e+00]
 [8.388608e-02 0.000000e+00 0.000000e+00 ... 0.000000e+00 0.000000e+00
  8.388608e-02]]


2 qubit 

In [48]:
ev = np.tensor(0.7781, requires_grad=False)  # observed expectation value

def sigmoid(x):
    return 1/(1+np.exp(-x))

@qml.qnode(dev)
def damping_circuit(x):
    qml.Hadamard(wires=0)
    qml.CNOT(wires=[0, 1])
    qml.AmplitudeDamping(sigmoid(x), wires=0)  # p = sigmoid(x)
    qml.AmplitudeDamping(sigmoid(x), wires=1)
    return qml.expval(qml.PauliZ(0) @ qml.PauliZ(1))

In [49]:

def cost(x, target):
    return (damping_circuit(x) - target)**2

In [50]:
opt = qml.GradientDescentOptimizer(stepsize=10)
steps = 35
x = np.tensor(0.0, requires_grad=True)

for i in range(steps):
    (x, ev), cost_val = opt.step_and_cost(cost, x, ev)
    if i % 5 == 0 or i == steps - 1:
        print(f"Step: {i}    Cost: {cost_val}")

print(f"QNode output after optimization = {damping_circuit(x):.4f}")
print(f"Experimental expectation value = {ev}")
print(f"Optimized noise parameter p = {sigmoid(x.take(0)):.4f}")

/usr/local/lib/python3.7/dist-packages/autograd/numpy/numpy_wrapper.py:156: ComplexWarning: Casting complex values to real discards the imaginary part
  return A.astype(dtype, order, casting, subok, copy)


Step: 0    Cost: 0.07733961000000007
Step: 5    Cost: 0.07733961000000007
Step: 10    Cost: 0.07733961000000007
Step: 15    Cost: 0.07733961000000007
Step: 20    Cost: 0.07733961000000007
Step: 25    Cost: 0.07733961000000007
Step: 30    Cost: 0.07733961000000007
Step: 34    Cost: 0.07733961000000007
QNode output after optimization = 0.5000
Experimental expectation value = 0.7781
Optimized noise parameter p = 0.5000


In [51]:
print(f"Output state is = \n{np.real(dev.state)}")

Output state is = 
[[0.625 0.    0.    ... 0.    0.    0.   ]
 [0.    0.    0.    ... 0.    0.    0.   ]
 [0.    0.    0.    ... 0.    0.    0.   ]
 ...
 [0.    0.    0.    ... 0.    0.    0.   ]
 [0.    0.    0.    ... 0.    0.    0.   ]
 [0.    0.    0.    ... 0.    0.    0.   ]]
